# Validation and intercomparison of AgERA5 and other reanalysis datasets for agricultural applications

Production date: DD-MM-2025

**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Dataset version: 2.0.

Produced by: C3S2_521 contract.

## 🌍 Use case: Agricultural yield estimation and prediction based on reanalysis data

## ❓ Quality assessment question
* Is AgERA5 fit-for-purpose as an input dataset for crop yield models?
* How do reanalysis datasets compare to observations, and to each other, for agriculturally relevant variables?

Introduction:

* Introduction to crop yield estimation -> [PCSE/WOFOST](https://github.com/ajwdewit/pcse) [[deWit+19]](https://doi.org/10.1016/j.agsy.2018.06.018).
* AgERA5 dataset, reanalysis in general
* Validation of datasets

[[CDS AgERA5]](https://doi.org/10.24381/cds.6c68c9bb).

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* The AgERA5 dataset ("Agrometeorological indicators from 1979 to present derived from reanalysis") is well-suited to agricultural studies because it provides daily aggregates and statistics of important parameters such as irradiation, temperature, vapour pressure, precipitation, wind speed, and snow depth. 
* The resolution of AgERA5 (daily, 0.1°) is comparable to other data sources and is sufficient for simulations at plot-level and larger spatial scales. Comparisons can be made at scales larger than the cell size, i.e. 0.1° or ~11 km.
* The AgERA5 and ERA5-Land datasets, both derived from ERA5, provide different values due to their different methods for downscaling and bias adjustment. While these differences are (in some cases) statistically significant, they are small compared to the typical uncertainty in ERA5 and compared to the uncertainties in agricultural models. Differences may be larger in specific areas, e.g. at higher elevation.
* etc
```

## 📋 Methodology

**Agrometeorological indicators from 1979 to present derived from reanalysis** (*AgERA5*; [doi 10.24381/cds.6c68c9bb](https://doi.org/10.24381/cds.6c68c9bb)).

A ‘free text’ introduction to the data analysis steps or a description of the literature synthesis, with a justification of the approach taken, and limitations mentioned. **Mention which CDS catalogue entry is used, including a link, and also any other entries used for the assessment**.

Variables of interest for a crop growth simulator such as [PCSE/WOFOST](https://github.com/ajwdewit/pcse):

| Variable name     | Statistic (24h) | Unit     | Example assessment |
|-------------------|-----------------|----------|--------------------|
| Solar irradiation | Total           | J/m²/day | example |
| Temperature (2m)  | Minimum         | °C       | example |
|                   | Maximum         |          |         |
|                   | Mean            |          |         |
| Vapour pressure   | Mean            | kPa      | example |
| Rain              | Total           | cm/day   | example |
| Wind speed (2m)   | Mean            | m/s      | example |
| Snow depth        | Mean            | cm       | example |

[Source](https://pcse.readthedocs.io/en/stable/code.html#pcse.base.WeatherDataContainer)
E0, ES0, ET0 are taken from evapotranspiration calculation

The analysis and results are organised in the following steps, which are detailed in the sections below:

**[](section-setup)**

**[](section-download)**
 * AgERA5, ERA5-Land, E-OBS, ...

**[](section-results)** 
 * Point-by-point comparison
 * Time series comparison
 * Geospatial comparison

Any further notes on the method could go here (explanations, caveats or limitations).

## 📈 Analysis and results

(section-setup)=
### 1. Code setup
#### Imports
```{note}
This notebook uses [earthkit](https://github.com/ecmwf/earthkit) for 
downloading ([earthkit-data](https://github.com/ecmwf/earthkit-data)) 
and 
visualising ([earthkit-plots](https://github.com/ecmwf/earthkit-plots)) data.
Because earthkit is in active development, some functionality may change after this notebook is published.
If any part of the code stops functioning, please raise an issue on our GitHub repository so it can be fixed.
```

In [ ]:
# Earthkit
import earthkit.data as ekd
import earthkit.plots as ekp

# General data handling
import numpy as np
import pandas as pd
import xarray as xr

# Visualisation
from matplotlib import pyplot as plt
from myst_nb import glue  # For web version

#### Helper functions

In [ ]:
# Type hints for helper functions
from typing import Callable, Optional, Iterable

# For pre-defining functions
from functools import partial

In [ ]:
## Data downloading
def domain_to_request(domain: ekp.geo.domains.Domain) -> dict:
    """ From an earthkit-plots domain, generate a request for earthkit-data / cdsapi. """
    bbox = domain.bbox.to_latlon_bbox()

    # Round
    north = int(np.ceil(bbox.north) + 1)
    south = int(np.floor(bbox.south) - 1)
    west = int(np.floor(bbox.west) - 1)
    east = int(np.ceil(bbox.east) + 1)
    
    area = [north, west, south, east]
    return {"area": area}

In [ ]:
## Data (pre-)processing
# Pre-defined variables to ensure consistent order
variables = ["Temperature_Air_2m_Min_24h", "Temperature_Air_2m_Mean_24h", "Temperature_Air_2m_Max_24h", 
             "Solar_Radiation_Flux", 
             "Wind_Speed_10m_Mean_24h",
             "Precipitation_Flux",
             "Snow_Thickness_Mean_24h"
            ]
variables_temperature = [var for var in variables if "Temperature" in var]
variables_not_temperature = [var for var in variables if var not in variables_temperature]

# Metadata handling
def adjust_metadata(data_array: xr.DataArray, **updates) -> xr.DataArray:
    """ Adjust metadata using a new dictionary. Returns a new object. """
    metadata_old = data_array.attrs
    metadata_new = metadata_old | updates
    data_array = data_array.assign_attrs(**metadata_new)
    return data_array

def adjust_names(dataset: xr.Dataset, dataset_name: str) -> xr.Dataset:
    """
    Adjust the names and units of pre-defined variables in a dataset.
    Also adds the name of the dataset to its attrs for easy acces.
    """
    # Rename variables
    dataset["Solar_Radiation_Flux"]        = adjust_metadata(dataset["Solar_Radiation_Flux"], 
                                                             long_name="Surface solar radiation downwards", units="J m-2 d-1")
    dataset["Temperature_Air_2m_Min_24h"]  = adjust_metadata(dataset["Temperature_Air_2m_Min_24h"], 
                                                             long_name="Minimum air temperature", units="°C")
    dataset["Temperature_Air_2m_Mean_24h"] = adjust_metadata(dataset["Temperature_Air_2m_Mean_24h"], 
                                                             long_name="Mean air temperature", units="°C")
    dataset["Temperature_Air_2m_Max_24h"]  = adjust_metadata(dataset["Temperature_Air_2m_Max_24h"], 
                                                             long_name="Maximum air temperature", units="°C")
    dataset["Wind_Speed_10m_Mean_24h"]     = adjust_metadata(dataset["Wind_Speed_10m_Mean_24h"], 
                                                             long_name="Wind speed at 10m", units="m s-1")
    dataset["Precipitation_Flux"]          = adjust_metadata(dataset["Precipitation_Flux"], 
                                                             long_name="Total precipitation", units="mm d-1")
    dataset["Snow_Thickness_Mean_24h"]     = adjust_metadata(dataset["Snow_Thickness_Mean_24h"], 
                                                             long_name="Snow depth", units="cm")
    
    # Assign dataset name
    dataset = dataset.assign_attrs({"name": dataset_name})
    return dataset

# Unit conversion
def convert_unit(dataset: xr.Dataset, key: str, conversion: Callable, new_unit: str) -> None:
    """ Convert the units of dataset[key] to new_unit using a conversion function (e.g. lambda x: x*1000 for m to mm), in-place. """
    # Metadata handling
    metadata_old = dataset[key].attrs
    metadata_new = metadata_old | {"units": new_unit}

    # Apply changes
    dataset[key] = conversion(dataset[key]).assign_attrs(**metadata_new)

convert_m2cm = partial(convert_unit, conversion=(lambda x: x*100), new_unit="cm")  # meter -> centimeter
convert_m2mm = partial(convert_unit, conversion=(lambda x: x*1000), new_unit="cm")  # meter -> millimeter
convert_K2C = partial(convert_unit, conversion=(lambda x: x-273.15), new_unit="°C")  # Kelvin -> Celsius

In [ ]:
## Time series
# Time series selection
def extract_timeseries(site: dict, *datasets: xr.Dataset, method: str="nearest", **kwargs) -> list[xr.Dataset]:
    """ Extract time series for a given site from a number of datasets. """
    selection = site | {"method": method} | kwargs
    timeseries = [dataset.sel(**selection) for dataset in datasets]
    return timeseries

In [ ]:
## Display
# Labels
def label_with_unit(data: xr.Dataset, key: str, *, latex=False) -> str:
    """ Extract the full name with unit for a key in a dataset. """
    long_name = data[key].long_name
    unit = data[key].units
    if latex:
        unit = ekp.metadata.units.format_units(data[key].units)
    return f"{long_name} [{unit}]"

# Plotting
plt.rcParams["grid.linestyle"] = "--"
nvars = len(variables)
nvars_half = int(np.ceil(nvars / 2))

def subplots_2byN(layout="constrained", **kwargs) -> tuple[plt.Figure, np.ndarray[plt.Axes]]:
    """
    Create a figure with 2 x (N/2) panels, with N the number of variables.
    Return them unravelled, turning off and removing any spares (e.g. 8th panel for 7 variables).
    """
    # Create figure, panels
    fig, axs = plt.subplots(nrows=nvars_half, ncols=2, layout=layout, **kwargs)
    axs = axs.ravel()

    # White out last panel if odd number of variables
    for ax in axs[nvars:]:
        ax.set_axis_off()
    axs = axs[:nvars]

    return fig, axs

In [ ]:
## Statistics
# Difference between datasets
def difference_between_datasets(dataset1: xr.Dataset, dataset2: xr.Dataset, *, 
                                diff_variables: Iterable[str]=variables) -> xr.Dataset:
    """ Calculate the difference between two datasets, preserving CRS and updating metadata. """
    # Subtract
    difference = dataset1 - dataset2

    # Adjust metadata
    for var in diff_variables:
        old_metadata = dataset1[var].attrs
        updated_metadata = old_metadata | {"long_name": r"Δ " + old_metadata["long_name"]}
        difference[var] = adjust_metadata(difference[var], **updated_metadata)

    # Add name
    name1, name2 = [dataset.name if hasattr(dataset, "name") else "<unspecified>" for dataset in (dataset1, dataset2)]
    difference = difference.assign_attrs({"name": f"Difference: {name1} – {name2}"})
        
    return difference

def relative_difference_between_datasets(dataset1: xr.Dataset, dataset2: xr.Dataset, *,
                                         diff_variables: Iterable[str]=variables, 
                                         exclude_substrings: Iterable[str]=["Temperature"]) -> xr.Dataset:
    """
    Calculate the relative [%] difference between two datasets, preserving CRS and updating metadata.
    Relative difference is calculated symmetrically, i.e. divided by (dataset1 + dataset2)/2.
    Where dataset1 == 0 and dataset2 == 0, the relative difference is set to 0 too.
    Variables can be excluded with exclude_substrings; e.g. don't divide temperatures.
    """
    # Select and calculate
    exclude_variables = [var for var in diff_variables if any(sub in var for sub in exclude_substrings)]
    diff_variables = [var for var in diff_variables if not var in exclude_variables]
    dataset1, dataset2 = [dataset.drop_vars(exclude_variables) for dataset in (dataset1, dataset2)]
    
    relative_difference = (dataset1 - dataset2) / (dataset1 + dataset2) * 200.

    # Replace 0/0 with 0
    both_zero = ((dataset1 + dataset2) <= 1e-5)  # Threshold slightly > 0 because of floating-point errors
    relative_difference = relative_difference.where(~both_zero, 0.)

    # Adjust metadata
    for var in diff_variables:
        old_metadata = dataset1[var].attrs
        updated_metadata = old_metadata | {"long_name": r"rΔ " + old_metadata["long_name"], "units": "%"}
        relative_difference[var] = adjust_metadata(relative_difference[var], **updated_metadata)

    # Add name
    name1, name2 = [dataset.name if hasattr(dataset, "name") else "<unspecified>" for dataset in (dataset1, dataset2)]
    relative_difference = relative_difference.assign_attrs({"name": f"% Difference: {name1} – {name2}"})

    return relative_difference
    

def difference_statistics(dataset1: xr.Dataset, dataset2: xr.Dataset, *,
                          diff_variables: Iterable[str]=variables) -> pd.DataFrame:
    """ Given two datasets, calculate a number of statistics for each variable and return the result in a table. """
    differences = difference_between_datasets(dataset1, dataset2, diff_variables=diff_variables)
    differences_rel = relative_difference_between_datasets(dataset1, dataset2, diff_variables=diff_variables)

    # Convert to pandas
    differences = differences.to_dataframe()
    differences_abs = differences.abs()

    differences_rel = differences_rel.to_dataframe()
    differences_rel_abs = differences_rel.abs()

    # Calculate aggregate statistics
    # Add quantiles?
    md = differences.agg(["median", "mean"])  \
                    .rename({"median": r"Median", "mean": "Mean"})
    mad = differences_abs.agg(["median"])  \
                         .rename({"median": r"Median Abs."})
    mapd = differences_rel_abs.agg(["median"])  \
                              .rename({"median": r"Median Abs. %"})
    stats = pd.concat([md, mad, mapd])
    stats = stats[diff_variables].T
    return stats

def display_difference_stats(dataset1: xr.Dataset, dataset2: xr.Dataset, **kwargs) -> str:
    """ Given two datasets, calculate a number of statistics for each variable and display the result in a table. """
    differences = difference_between_datasets(dataset1, dataset2, **kwargs)
    difference_stats = difference_statistics(dataset1, dataset2, **kwargs)
    formatted = difference_stats.style \
                                .format(precision=3)  \
                                .set_caption(differences.name)  \
                                .relabel_index([label_with_unit(differences, var) for var in difference_stats.index])
    return formatted

In [ ]:
## Plotting functions
# Colour maps
def find_percentile(*data_arrays: Iterable[xr.DataArray], percentile: float, round: str=None) -> float:
    """
    Find the specified percentile across all of the provided datasets.
    Used for making consistent colour maps.
    """
    data_flat = np.concatenate([arr.to_numpy().ravel() for arr in data_arrays])
    perc = np.nanpercentile(data_flat, percentile)
    if round == "up":
        perc = np.ceil(perc)
    elif round == "down":
        perc = np.floor(perc)
    return perc

cmap_percentile = 0.5
find_vmin = partial(find_percentile, percentile=cmap_percentile)
find_vmax = partial(find_percentile, percentile=100-cmap_percentile)

(section-download)=
### 2. Download data
#### General setup
This notebook uses [earthkit-data](https://github.com/ecmwf/earthkit-data) to download files from the CDS.
If you intend to run this notebook multiple times, it is highly recommended that you [enable caching](https://earthkit-data.readthedocs.io/en/latest/guide/caching.html) to prevent having to download the same files multiple times.

We will be downloading multiple datasets in this notebook.
In this section, we define the parameters common to all datasets: time and space.
This way, these only need to be changed in one place if you wish to modify the notebook for your own use case.

In this example, we will be looking at data for the United Kingdom and Ireland every day in January–September 2023.
We will also do a time series comparison in one site within this area.
These settings can easily be changed by editing the variables in the following cell:

In [ ]:
domain = ekp.geo.domains.union(["United Kingdom", "Ireland"], name="UK & Ireland")
site = {"lat": 52.5, "lon": 0.0}
year = 2023

We now set up default CDS request parameters based on these variables:

In [ ]:
request_domain = domain_to_request(domain)

request_time = {
    "year": year,
    "month": [f"{mo:02}" for mo in range(1, 10)],
    "day": [f"{d:02}" for d in range(1, 32)],
}

We can define a helper function that adds the time and domain parameters, as well as a dictionary of parameters specific to one dataset (e.g. AgERA5, ERA5-Land), to a number of requests:

In [ ]:
def make_full_request(request_dataset: dict, *requests: dict) -> dict:
    """ Combine default requests (time + domain) with a dataset-specific default request (request_dataset) and any number of individual (e.g. variable-specific) requests. """
    base_request = request_time | request_domain | request_dataset
    updated_requests = [base_request | req for req in requests]
    return updated_requests

#### AgERA5
We now define parameters unique to AgERA5:

In [ ]:
agera5_ID = "sis-agrometeorological-indicators"

request_agera5 = {
    "version": "2_0",
}

Next, we specify the variables of interest:

In [ ]:
request_irradiation = {
    "variable": "solar_radiation_flux",
}

# Temperature has to be split into separate requests because of size limits
request_temperature_min = {
    "variable": "2m_temperature",
    "statistic": ["24_hour_minimum"],
}

request_temperature_max = {
    "variable": "2m_temperature",
    "statistic": ["24_hour_maximum"],
}

request_temperature_mean = {
    "variable": "2m_temperature",
    "statistic": ["24_hour_mean"],
}

request_rain = {
    "variable": "precipitation_flux",
}

request_wind = {
    "variable": "10m_wind_speed",
    "statistic": ["24_hour_mean"],
}

request_snow = {
    "variable": "snow_thickness",
    "statistic": ["24_hour_mean"],
}

The requests for specific variables are combined with the default, time, and domain parameters and passed to earthkit for download from the CDS:

In [ ]:
requests_agera5_combined = make_full_request(request_agera5,
                                             request_irradiation,
                                             request_temperature_min, request_temperature_max, request_temperature_mean,
                                             request_rain,
                                             request_wind,
                                             request_snow,
                                            )

ds_agera5 = ekd.from_source("cds", agera5_ID, *requests_agera5_combined)

Earthkit-data downloads the dataset as a [field list](https://earthkit-data.readthedocs.io/en/latest/guide/data.html), which can be manipulated directly.
Here, we convert it to an Xarray object for ease of use later (when comparing multiple datasets):

In [ ]:
print("AgERA5 data type from earthkit-data:", type(ds_agera5))
data_agera5 = ds_agera5.to_xarray(compat="equals")
print("AgERA5 data type in Xarray:", type(data_agera5))
data_agera5

#### ERA5-Land
We now define parameters unique to ERA5-Land and the variables of interest.
This will involve two CDS datasets.
The [*ERA5-Land hourly data from 1950 to present*](https://doi.org/10.24381/cds.e2161bac) dataset contains hourly data for variables like 2m temperature as well as accumulated data for variables like solar irradiation.
For the present use case, we are only interested in the daily accumulated data.
[*ERA5-Land post-processed daily statistics from 1950 to present*](https://doi.org/10.24381/cds.e9c9c792) provides daily minimum/maximum/mean statistics for the hourly variables, saving us the effort of aggregating these ourselves.
In the following subsections, we will download the data (accumulated and daily statistics) and harmonise these to the same format as AgERA5.

##### Accumulated data from ERA5-Land
Per the [documentation for ERA5-Land](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation#heading-Accumulations):
The accumulations in the short forecasts of ERA5-Land (with hourly steps from 01 to 24) are treated the same as those in ERA-Interim or ERA-Interim/Land, i.e., they are accumulated from the beginning of the forecast to the end of the forecast step. For example, runoff at day=D, step=12 will provide runoff accumulated from day=D, time=0 to day=D, time=12. The maximum accumulation is over 24 hours, i.e., from day=D, time=0 to day=D+1,time=0 (step=24). For the CDS time, or validity time, of 00 UTC, the accumulations are over the 24 hours ending at 00 UTC i.e. the accumulation is during the previous day.

In practice, this means that we need to download data for *day+1* (e.g. 2 January 2024) to get the total accumulated value for *day* (1 January 2024).
Our existing `request_time` will provide data for 2023-12-31 – 2024-09-29, so we need to add one extra day.
This is achieved by adding a second request for just the last day to the earthkit-data download.

In [ ]:
era5land_ID = "reanalysis-era5-land"

request_era5land = {
    "time": ["00:00"],
    "data_format": "grib",
    "download_format": "unarchived",
}

request_era5land_extratime = {
    "month": "10",
    "day": "01",
}

In [ ]:
request_irradation = {
    "variable": ["surface_solar_radiation_downwards"],
}

request_rain = {
    "variable": ["total_precipitation"],
}

In [ ]:
request_era5land_irradiation, request_era5land_rain = make_full_request(request_era5land,
                                                                        request_irradation,
                                                                        request_rain,
                                                                       )

ds_era5land_accumulated = ekd.from_source("cds", era5land_ID, request_era5land_irradiation, request_era5land_irradiation | request_era5land_extratime,
                                                              request_era5land_rain, request_era5land_rain | request_era5land_extratime)
data_era5land_accumulated = ds_era5land_accumulated.to_xarray()

When we inspect the resulting dataset (in Xarray format), we see that the `forecast_reference_time` coordinate conveniently matches the variable to the day of accumulation:

In [ ]:
data_era5land_accumulated

##### Daily statistics from the post-processed ERA5-Land dataset
The daily statistics are indexed according to the day they apply to, meaning we do not need to worry about adding extra days here.

In [ ]:
era5land_ID = "derived-era5-land-daily-statistics"

request_era5land = {
    "time_zone": "utc+00:00",
    "frequency": "1_hourly",
}

In [ ]:
# Temperature has to be split into separate requests because of size limits
request_temperature_min = {
    "variable": "2m_temperature",
    "daily_statistic": "daily_minimum",
}

request_temperature_max = {
    "variable": "2m_temperature",
    "daily_statistic": ["daily_maximum"],
}

request_temperature_mean = {
    "variable": "2m_temperature",
    "daily_statistic": ["daily_mean"],
}

# request_vapour_pressure : Not available

request_wind_u = {
    "variable": "10m_u_component_of_wind",
    "daily_statistic": ["daily_mean"],
}

request_wind_v = {
    "variable": "10m_v_component_of_wind",
    "daily_statistic": ["daily_mean"],
}

request_snow = {
    "variable": "snow_depth",
    "daily_statistic": ["daily_mean"],
}

We download the different variables separately in anticipation of the harmonisation step in the next subsection:

In [ ]:
requests_era5land_combined = make_full_request(request_era5land,
                                               request_temperature_min, request_temperature_max, request_temperature_mean,
                                               request_wind_u, request_wind_v,
                                               request_snow,
                                              )

ds_era5land = [ekd.from_source("cds", era5land_ID, req) for req in requests_era5land_combined]
data_era5land = [ds.to_xarray() for ds in ds_era5land]
data_era5land_temperature_min, data_era5land_temperature_max, data_era5land_temperature_mean, data_era5land_wind_u, data_era5land_wind_v, data_era5land_snow = data_era5land

##### Harmonisation
The ERA5-Land dataset is structured differently from AgERA5 and requires some processing before the two can be intercompared.
This involves renaming coordinates and variables, and adjusting units.

For the accumulated data, we need to rename the variables and coordinates to match those in AgERA5, and convert the units for precipitation to mm:

In [ ]:
data_era5land_accumulated = data_era5land_accumulated.rename({"ssrd": "Solar_Radiation_Flux",
                                                              "tp": "Precipitation_Flux",
                                                              "forecast_reference_time": "time", 
                                                              "latitude": "lat", "longitude": "lon"})

convert_m2mm(data_era5land_accumulated, "Precipitation_Flux")

The temperature statistics are downloaded as simply `t2m`.
These need to be renamed before they can be combined:

In [ ]:
data_era5land_temperature_min = data_era5land_temperature_min.rename({"t2m": "Temperature_Air_2m_Min_24h"})
data_era5land_temperature_max = data_era5land_temperature_max.rename({"t2m": "Temperature_Air_2m_Max_24h"})
data_era5land_temperature_mean = data_era5land_temperature_mean.rename({"t2m": "Temperature_Air_2m_Mean_24h"})
data_era5land_temperature = xr.merge([data_era5land_temperature_min, data_era5land_temperature_max, data_era5land_temperature_mean], compat="equals")

The 10 m wind speed is calculated from the two variables representing its U (east–west) and V (north–south) components:

In [ ]:
data_era5land_wind = xr.merge([data_era5land_wind_u, data_era5land_wind_v], compat="equals")
data_era5land_wind = data_era5land_wind.assign(
    {"Wind_Speed_10m_Mean_24h": xr.ufuncs.sqrt(data_era5land_wind["u10"]**2 + data_era5land_wind["v10"]**2)}
)

Lastly, we rename the snow variable and change its unit to match AgERA5:

In [ ]:
data_era5land_snow = data_era5land_snow.rename({"sde": "Snow_Thickness_Mean_24h"})
convert_m2cm(data_era5land_snow, "Snow_Thickness_Mean_24h")

Now we can combine the pre-processed variables into a single Xarray dataset.
We also rename the coordinates to match AgERA5.

In [ ]:
data_era5land = xr.merge([data_era5land_temperature, data_era5land_wind, data_era5land_snow], compat="equals")
data_era5land = data_era5land.rename({"valid_time": "time", "latitude": "lat", "longitude": "lon"})

Lastly, we combine the pre-processed and accumulated variables:

In [ ]:
data_era5land = xr.merge([data_era5land_accumulated, data_era5land], compat="equals")
data_era5land

#### E-OBS
We now define parameters unique to E-OBS and the variables of interest:

In [ ]:
eobs_ID = "insitu-gridded-observations-europe"
request_eobs = {
    "product_type": "ensemble_mean",
    "variable": [
        "mean_temperature",
        "minimum_temperature",
        "maximum_temperature",
        "precipitation_amount",
        "surface_shortwave_downwelling_radiation",
        "wind_speed"
    ],
    "grid_resolution": "0_1deg",
    "period": "2011_2024",
    "version": ["30_0e"]
}

In [ ]:
ds_eobs = ekd.from_source("cds", eobs_ID, request_eobs | request_domain)
data_eobs = ds_eobs.to_xarray(compat="equals")

In [ ]:
eobs_ID = "insitu-gridded-observations-europe"
request_eobs = {
    "product_type": "ensemble_mean",
    "variable": [
        "mean_temperature",
    ],
    "grid_resolution": "0_1deg",
    "period": "2011_2024",
    "version": ["30_0e"]
}

In [ ]:
# Do individually to avoid xr error?
ds_eobs = ekd.from_source("cds", eobs_ID, request_eobs | request_domain)
data_eobs = ds_eobs.to_xarray(compat="equals")

Rename variables and coordinates

In [ ]:
data_eobs

#### General harmonisation
##### Grid alignment
Before the data can be analysed, we must make sure their coordinates are aligned equally.
All of the datasets used here are provided on a regular 0.1° by 0.1° grid, so no regridding is necessary.
However, two steps need to be taken before the data can be compared directly:
* Their representation as floating-point numbers can cause very small differences to appear, which do not reflect any real differences in the data but are difficult for software (in this case Xarray) to work with. Knowing that the data are on a regular 0.1° by 0.1° grid, we can simply round all of the coordinates to 2 digits to force them to be the same.
* The bounds of the datasets (in space and in time) are slightly different and need to be aligned.

In [ ]:
datasets = [data_agera5, data_era5land]

In [ ]:
# Round coordinates before alignment to avoid floating-point errors
# Cannot be a dict-comp with coord in lat/lon because of symbol table issues
d = 2
round_mapping = {"lat": (lambda dataset: dataset["lat"].round(d)),
                 "lon": (lambda dataset: dataset["lon"].round(d))}

datasets_rounded = [dataset.assign_coords(round_mapping) for dataset in datasets]

In [ ]:
# Align data using an inner join
data_agera5, data_era5land, data_eobs = xr.align(*datasets_rounded, data_eobs, join="inner")

##### Units and variable names
We also convert the temperatures from Kelvin to degrees Celsius, which are more commonly used in agricultural studies:

In [ ]:
for variable in variables_temperature:
    convert_K2C(data_agera5, variable)
    convert_K2C(data_era5land, variable)

Lastly, we harmonise the variable "long" names and units, as stored in xarray metadata, to simplify the analysis steps and figures later on.
This is not strictly necessary, but it is convenient.

In [ ]:
data_agera5 = adjust_names(data_agera5, "AgERA5")
data_era5land = adjust_names(data_era5land, "ERA5-Land")

(section-results)=
### 3. Results
Describe what is done in this step/section and what the `code` in the cell does (if code is included). 

If this is the **results section**, we expect the final plots to be created here with a description of how to interpret them, and what information can be extracted for the specific use case and user question. The information in the 'quality assessment statement' should be derived here.

#### Point-by-point comparison
Here, we compare the different variables 1-to-1 between the different datasets, across their entire spatial and temporal domain, to determine the typical differences.

We start with a histogram of the 1-to-1 differences as well as the corresponding statistics: median absolute deviation (MAD), median bias.

In [ ]:
differences = difference_between_datasets(data_agera5, data_era5land)
display_difference_stats(data_agera5, data_era5land)

In [ ]:
# Create figure
fig, axs = subplots_2byN(figsize=(8, 8))

# Plot individual scatter plots
for ax, var in zip(axs, variables):
    diff = differences[var]

    # Symmetric x axis
    low, high = find_vmin(diff), find_vmax(diff)
    lim = max(abs(low), abs(high))
    bins = np.linspace(-lim, lim, 500)

    # Plot differences
    try:  # Account for missing variables
        ax.hist(diff.values.ravel(),
                bins=bins, color="black")
    except KeyError:
        print(f"KeyError: no variable `{var}` in one of the datasets")
    
    # X-axis settings
    ax.set_xlabel(label_with_unit(differences, var, latex=True))
    ax.set_xlim(-lim, lim)

# Visual settings
for ax in axs:
    grid = ax.grid(True, axis="both")
    ax.set_title("")

    # Highlight 0
    ax.axvline(0, color=plt.rcParams["grid.color"], linewidth=1.5, linestyle="-")

fig.suptitle(f"{differences.name} (overall distribution)")

# Uncomment if running this notebook yourself:
# Show result
plt.show()

# Uncomment if building the Jupyter-book web version:
glue("fig_histogram", fig, display=False)
plt.close()

```{glue:figure} fig_histogram
:figwidth: 700px
:name: "fig_histogram"

Distributions of point-by-point differences between AgERA5 and ERA5-Land for each variable.
```

We can use a scatter density plot to look for patterns in the 1-to-1 comparisons:

In [ ]:
# Create figure
fig, axs = subplots_2byN(figsize=(10, 20))

# Plot individual scatter plots
for ax, var in zip(axs, variables):
    try:  # Account for missing variables
        ax.hexbin(data_agera5[var].values.ravel(), data_era5land[var].values.ravel(),
                  gridsize=500, mincnt=1, cmap="cividis")
    except KeyError:
        print(f"KeyError: no variable `{var}` in one of the datasets")

    ax.text(0.05, 0.95, label_with_unit(data_agera5, var, latex=True), 
            horizontalalignment="left", verticalalignment="top", transform=ax.transAxes,
            bbox={"facecolor": "white", "edgecolor": "black", "alpha": 1})

# Visual settings
for ax in axs:
    ax.grid(True, axis="both")
    ax.set_aspect("equal", "box")
    ax.set_xlabel(data_agera5.name)
    ax.set_ylabel(data_era5land.name)
    ax.set_title("")

    # Highlight diagonal
    ax.axline((0, 0), slope=1, color=plt.rcParams["grid.color"], linewidth=1.5, linestyle="-")

fig.suptitle(f"Point-by-point comparison")

# Uncomment if running this notebook yourself:
# Show result
plt.show()

# Uncomment if building the Jupyter-book web version:
glue("fig_scatter", fig, display=False)
plt.close()

```{glue:figure} fig_scatter
:figwidth: 700px
:name: "fig_scatter"

Point-by-point comparison (density scatter) between AgERA5 and ERA5-Land for each variable.
```

#### Time series comparison
Crop yield estimates make use of the time series of the relevant variables in a given site.
Therefore, a comparison between time series from the different datasets should provide a better estimate of the differences downstream than an overall point-by-point comparison.

First, we select data at our test site (defined at the top of this notebook) and display the differences between the time series from the different datasets:

In [ ]:
timeseries_agera5, timeseries_era5land = extract_timeseries(site, data_agera5, data_era5land)

In [ ]:
display_difference_stats(timeseries_agera5, timeseries_era5land)

We now create a plot showing all variables:

In [ ]:
# Create figure
fig, axs = plt.subplots(nrows=nvars, figsize=(10, 20), sharex=True, layout="constrained")

# Plot individual time series
for j, timeseries in enumerate([timeseries_agera5, timeseries_era5land]):
    for ax, var in zip(axs, variables):
        try:  # Account for missing variables
            timeseries[var].plot(ax=ax, alpha=0.7)
        except KeyError:
            print(f"KeyError: no variable `{var}` in dataset {j}")

        ax.set_ylabel(label_with_unit(timeseries, var, latex=True))

# Visual settings
for ax in axs:
    ax.grid(True, axis="both")
    ax.tick_params("x", labelbottom=True)
    ax.set_xlabel("")
    ax.set_title("")

    ax.axhline(0, color=plt.rcParams["grid.color"], linewidth=1.5, linestyle="-")

fig.suptitle(f"Time series comparison at ({site['lat']} °N, {site['lon']} °E)")

# Uncomment if running this notebook yourself:
# Show result
plt.show()

# Uncomment if building the Jupyter-book web version:
glue("fig_timeseries", fig, display=False)
plt.close()

```{glue:figure} fig_timeseries
:figwidth: 700px
:name: "fig_timeseries"

Time series comparison between AgERA5 and ERA5-Land for each variable, in one site.
```

#### Geospatial comparison
In this section, we visually compare the datasets in space on a single day, using earthkit-plots.
This requires a bit of setup, namely defining the domain (time, space, datasets) and defining some plot parameters.

In [ ]:
# Define domain for plot (time, space, datasets)
date = f"{year}0101"  # 1 January
datasets_singleday = [dataset.sel(time=date) for dataset in (data_agera5, data_era5land)]

In [ ]:
## Define styles for earthkit-plots
# Temperature: Share between min/mean/max
temperature_data = [dataset[key] for key in variables_temperature for dataset in datasets_singleday]
temperature_vmin = find_vmin(*temperature_data, round="down")
temperature_vmax = find_vmax(*temperature_data, round="up")
style_temperature = ekp.styles.Style(
    levels = np.arange(temperature_vmin, temperature_vmax+2, 2),
    extend = "both",
)

# Surface radiation: Start at 0, in steps of 0.5 x 10^(order)
ceil_half = lambda x: np.ceil(x * 2) / 2
surface_radiation_data = [dataset["Solar_Radiation_Flux"] for dataset in datasets_singleday]
surface_radiation_vmax = find_vmax(*surface_radiation_data)
order = int(np.log10(surface_radiation_vmax))
vmax_nomag = surface_radiation_vmax / 10**order
surface_radiation_vmax = ceil_half(vmax_nomag) * 10**order
style_surface_radiation = ekp.styles.Style(
    levels = np.arange(0, surface_radiation_vmax+0.1, 0.5 * 10**order),
    extend = "max",
)

# Precipitation: Start at 0
precipitation_data = [dataset["Precipitation_Flux"] for dataset in datasets_singleday]
precipitation_vmax = find_vmax(*precipitation_data, round="up")
style_precipitation = ekp.styles.Style(
    levels = np.arange(0, precipitation_vmax, 2),
    extend = "max",
)

# Wind speed: Start at 0
wind_data = [dataset["Wind_Speed_10m_Mean_24h"] for dataset in datasets_singleday]
wind_vmax = find_vmax(*wind_data, round="up")
style_wind = ekp.styles.Style(
    levels = np.arange(0, wind_vmax, 1),
    extend = "max",
)

# Snow depth: Start at 0
snow_data = [dataset["Snow_Thickness_Mean_24h"] for dataset in datasets_singleday]
snow_vmax = find_vmax(*snow_data, round="up")
style_snow = ekp.styles.Style(
    levels = np.arange(0, snow_vmax, 2),
    extend = "both",
)

## Link styles to variable names
style_dict = {
    variable: style_temperature for variable in variables_temperature
} | {  # Combine temperature variables (all the same) with others (all different)
    "Solar_Radiation_Flux": style_surface_radiation,
    "Precipitation_Flux": style_precipitation,
    "Wind_Speed_10m_Mean_24h": style_wind,
    "Snow_Thickness_Mean_24h": style_snow,
}

In [ ]:
# Create figure
fig = ekp.Figure(rows=nvars, columns=2, size=(6, 20))

# Plot variables
for row, variable in enumerate(variables):
    # Get style from dictionary
    style = style_dict[variable]

    # Plot individual datasets
    for col, dataset in enumerate(datasets_singleday):
        subplot = fig.add_map(domain=domain)
        try:  # Account for missing variables
            subplot.grid_cells(dataset, z=variable, style=style)
        except KeyError:
            print(f"KeyError: no variable `{variable}` in dataset {col}")

        # Text
        ax = subplot.ax
        ax.text(0.05, 0.95, dataset.name, 
                horizontalalignment="left", verticalalignment="top", transform=ax.transAxes,
                bbox={"facecolor": "white", "edgecolor": "black", "alpha": 1})

    # Colour bar on the right (last ax)
    subplot.legend(label="{variable_name} [{units:~F}]", location="right")

# Decorate figures
fig.land()
fig.coastlines()
fig.gridlines(linestyle=plt.rcParams["grid.linestyle"])
fig.title("Geospatial intercomparison: {time:%-d %B %Y}")

# Uncomment if running this notebook yourself:
# Show result
plt.show()

# Uncomment if building the Jupyter-book web version:
fig = fig.fig  # Extract matplotlib fig from earthkit fig
glue("fig_geo", fig, display=False)
plt.close()

```{glue:figure} fig_geo
:figwidth: 700px
:name: "fig_geo"

Geospatial comparison between AgERA5 and ERA5-Land.
```

Note that differences in snow thickness are highest in areas where you wouldn't generally have crops anyway

## ℹ️ If you want to know more

### Key resources
The CDS catalogue entries for the data used were:
* Agrometeorological indicators from 1979 to present derived from reanalysis (AgERA5): [sis-agrometeorological-indicators](https://doi.org/10.24381/cds.6c68c9bb)
* ERA5-Land hourly data from 1950 to present: [reanalysis-era5-land](https://doi.org/10.24381/cds.e2161bac)
* ERA5-Land post-processed daily statistics from 1950 to present: [derived-era5-land-daily-statistics](https://doi.org/10.24381/cds.e9c9c792)

Code libraries used:
* [earthkit](https://github.com/ecmwf/earthkit)
  * [earthkit-data](https://github.com/ecmwf/earthkit)
  * [earthkit-plots](https://github.com/ecmwf/earthkit-plots)

More about crop yield estimation:
* [A gentle introduction to WOFOST](https://www.wur.nl/en/show/a-gentle-introduction-to-wofost.htm)
* [Crop yield prediction based on reanalysis and crop phenology data in the agroclimatic zones](https://doi.org/10.1007/s00704-024-05046-x)
* [Historical trends and future projections of compound cloudy-rainy events during the global winter wheat harvest phase](https://doi.org/10.1016/j.agrformet.2025.110637)

More about reanalysis data:
* [The ERA5 global reanalysis](https://doi.org/10.1002/qj.3803)
* [ERA5-Land: a state-of-the-art global reanalysis dataset for land applications](https://doi.org/10.5194/essd-13-4349-2021)
* AgERA5
  * [Algorithm Theoretical Basis (ATBD)](https://confluence.ecmwf.int/pages/viewpage.action?pageId=278550984)
  * [Product User Guide and Specification (PUGS)](https://confluence.ecmwf.int/pages/viewpage.action?pageId=278551004)
  * [Global Agriculture Downscaling and bias correction](https://confluence.ecmwf.int/display/CKB/Global+Agriculture+Downscaling+and+bias+correction)
  * [AgERA5tools Python package](https://github.com/ajwdewit/agera5tools)

Validation case studies for AgERA5:
* [AgERA5, MERRA-2 in Guinea-Bissau](https://doi.org/10.3390/hydrology12070161)
* [AgERA5, ERA5, CHIRP/CHIRPS, ENACTS, TAMSAT, PERSIANN-CDR/PERSIANN-CCS-CDR in Ghana and Zambia](https://doi.org/10.1007/s00704-025-05462-7)
* [AgERA5, ERA5-Land, E-OBS, CHELSA-W5E5, MSWEP, CHIRPS05, IMERG V06 in Greece](https://doi.org/10.1016/j.atmosres.2024.107888)
* [AgERA5, ERA5-Land, MERRA-2, PERSIANN-CDR in Peru](https://doi.org/10.1590/2318-0331.302520240068)
* [AgERA5, CHIRPS, PERSIANN in Sri Lanka](https://doi.org/10.1109/MERCon63886.2024.10689062)

### References
[[CDS AgERA5]](https://doi.org/10.24381/cds.6c68c9bb) Boogaard, H., Schubert, J., De Wit, A., Lazebnik, J., Hutjes, R., Van der Grijn, G., (2020): Agrometeorological indicators from 1979 to present derived from reanalysis. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). DOI: 10.24381/cds.6c68c9bb (Accessed on DD-MMM-YYYY)